In [2]:
import numpy as np
from scipy import stats
import os
import pandas as pd
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
import math
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import sqlite3
%matplotlib inline

In [4]:
data_remove_e = pd.read_csv("data_for_rohan.csv",low_memory=False)

In [12]:
data_remove_e = data_remove_e.drop('Unnamed: 0', axis=1)

In [14]:
data_remove_rpl = data_remove_e.drop(['COMPL_RPY_1YR_RT', 'NONCOM_RPY_1YR_RT', 'LO_INC_RPY_1YR_RT', 'MD_INC_RPY_1YR_RT'\
                                      , 'HI_INC_RPY_1YR_RT', 'DEP_RPY_1YR_RT', 'IND_RPY_1YR_RT', 'PELL_RPY_1YR_RT', \
                                      'NOPELL_RPY_1YR_RT', 'FEMALE_RPY_1YR_RT', 'MALE_RPY_1YR_RT', 'FIRSTGEN_RPY_1YR_RT', \
                                      'NOTFIRSTGEN_RPY_1YR_RT'], axis=1)

In [19]:
nan_counts = data_remove_rpl.count(axis=1)
proportion_nan = []
proportion_nan[:] = [1 - ((float) (a) / (float) (len(data_remove_rpl.columns))) for a in nan_counts]

In [30]:
df_p_nan = pd.DataFrame(proportion_nan)
delete_these_rows = df_p_nan.ix[df_p_nan[0] > 0.9].index.values
data_thinner = data_remove_rpl.drop(data_remove_rpl.index[delete_these_rows])
data_thinner.shape

(1729, 245)

In [ ]:
data_thinner.to_csv('data_to_impute.csv', encoding)